In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
import numpy as np

train_data_path = '/content/drive/MyDrive/train.csv'
test_data_path = '/content/drive/MyDrive/test.csv'

train_df = pd.read_csv(train_data_path)
test_df = pd.read_csv(test_data_path)


features = ["GrLivArea", "BedroomAbvGr", "FullBath", "HalfBath", "GarageArea"]
target = "SalePrice"

train_df = train_df.dropna(subset=features + [target])
test_df = test_df.dropna(subset=features)


X_train = train_df[features].values
y_train = train_df[target].values.reshape(-1, 1)


X_test = test_df[features].values


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)


class HousePriceModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(HousePriceModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.linear(x)


input_dim = X_train.shape[1]
output_dim = 1
model = HousePriceModel(input_dim, output_dim)


criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)


epochs = 500
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()


    predictions = model(X_train_tensor)
    loss = criterion(predictions, y_train_tensor)


    loss.backward()
    optimizer.step()


    if (epoch+1) % 50 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')


model.eval()
with torch.no_grad():
    test_predictions = model(X_test_tensor).numpy()


print("Predicted prices for test data:")
print(test_predictions[:10])


def predict_house_price(gr_liv_area, bedroom_abv_gr, full_bath, half_bath, garage_area):

    input_data = np.array([[gr_liv_area, bedroom_abv_gr, full_bath, half_bath, garage_area]])
    input_data = scaler.transform(input_data)
    input_tensor = torch.tensor(input_data, dtype=torch.float32)


    model.eval()
    with torch.no_grad():
        price_prediction = model(input_tensor).item()
    return price_prediction


def get_user_input_and_predict():
    print("Enter the following details to predict house price:")
    gr_liv_area = float(input("Above-ground living area (sq ft): "))
    bedroom_abv_gr = int(input("Number of bedrooms above ground: "))
    full_bath = int(input("Number of full bathrooms: "))
    half_bath = int(input("Number of half bathrooms: "))
    garage_area = float(input("Garage area (sq ft): "))

    predicted_price = predict_house_price(gr_liv_area, bedroom_abv_gr, full_bath, half_bath, garage_area)
    print(f"Predicted House Price: ${predicted_price:.2f}")


get_user_input_and_predict()


Epoch [50/500], Loss: 7023198208.0000
Epoch [100/500], Loss: 2916203264.0000
Epoch [150/500], Loss: 2343848448.0000
Epoch [200/500], Loss: 2250706688.0000
Epoch [250/500], Loss: 2228309504.0000
Epoch [300/500], Loss: 2219433472.0000
Epoch [350/500], Loss: 2214773248.0000
Epoch [400/500], Loss: 2212097024.0000
Epoch [450/500], Loss: 2210524672.0000
Epoch [500/500], Loss: 2209596416.0000
Predicted prices for test data:
[[161383.97]
 [134177.1 ]
 [201881.83]
 [198427.8 ]
 [190405.94]
 [199165.98]
 [153273.98]
 [178033.16]
 [177124.69]
 [136491.61]]
Enter the following details to predict house price:
